In [ ]:
// Importieren der benötigten Module
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE, 0.0.1"
#r "nuget: FSharp.Stats"

// ... und öffnen der benöätigten Module
open FsODE
open Plotly.NET
open FSharp.Stats


// der Rechenalgorithmus der Programmbibliothek, die wir benutzen:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

Installed Packages FSharp.Stats, 0.4.7 FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

### Einfaches Model vom Lac Operon

Einleitender Text der das Lac Operon erklärt








Um die komplexität des Lac Operons zu verringerin wird sich im folgenden nur die Konzentration von intrazellulärer Lactose angeschaut. Damit diese betrachtung möglichst einfach bleibt wird davon ausgegangen das es genug extrazeluläre Lactose gibt. Lactose kann wie oben beschrieben nur durch $\beta -\textrm {Galactosid-Permease}$ in die Zelle transportiert werden. LacY, das Gen das $\beta -\textrm {Galactosid-Permease}$ codiert, wird kontinuierlich expremiert auch wenn es durch LacI inhibiert wird. Allerdings ist die produzierte Menge an $\beta -\textrm {Galactosid-Permease}$ sehr gering. Das heißt Lactose kann immer in die Zelle transporitert werden, aber nur in gerningen Mengen solange LacI noch an den Promotor des Lac Operons bindet. <br>
Wenn die Konzentration in der Zelle hoch genug ist, kommt es wie besprochen zur Bildung von Allolactose welches eine Bindung mit LacI eingeht. Dies führt dazu das mehr $\beta -\textrm {Galactosid-Permease}$ produziert wird und somit mehr Lactose in die Zelle transportiert werden kann. <br>
Die Differenzialgleichung für die Konzentration von intrazellulärer Lactose kann somit folgendermaßen beschrieben werden. Es gibt einen unregulierten geringen Lactoseimport, genannt $\alpha$. Dieser ist nicht reguliert. Es gibt außerdem einen regulierten Lactoseimport, dieser wird $\beta$ genannt analog zur Proteinproduktion von $\beta -\textrm {Galactosid-Permease}$ die nötig ist damit dieser regulierte Transport beginnt. Da mehr intrazelluäre Lactose zu mehr Import führt, kann hier von einer postiven Autoregulation gesprochen werden. Abschliesend wird Lactose natürlich auch von der Zelle verbraucht, wobei die Menge die genutzt wird wieder von der Lactosekonzentration abhängt. Die Differenzialgleichung ist somit <br>

$$ Lactosekonzentration_{intrazellulär}' = \frac {dLactosekonzentration_{intrazellulär}} {dt}= \alpha + \beta * \frac {Lactosekonzentration_{intrazellulär}}{K_{D} + Lactosekonzentration_{intrazellulär}} - \gamma*Lactosekonzentration_{intrazellulär}$$
Da die Simulationen nur auf die richtige Differenzialgleichung achten, aber nicht auf die genutzten Namen der Parameter verkürzen wir die Gleichung zu
$$ inlac' = \frac {dinlac} {dt}= \alpha + \beta * \frac {inlac}{K_{D} + inlac} - \gamma*inlac$$

Aufgabe 4.1: <br>
Setzen sie die Gleichung für intrazelluläre Lactosekonzentration in die nachfolgende Simulation ein (Bitte beachten sie, dass sie die Lactosekonzentration als "inlac" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\alpha$, $\beta \textrm{ und } \gamma$ als "alpha", "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration

        //Differenzialgleichung die diese Änderung beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha + beta * (inlac)/((inlac) + (K_d)) - (gamma*inlac)

        [| inlac' |]

//Startkonzentrationen
let P0 = [| 
    0.1
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = Ps |> SolPoints.toPoints 1 
    lactose |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe 4.2: <br>
Varrieren sie die Startkonzentration. Was beobachten sie? Ist dies das Verhalten was sie in einer Zelle erwarten würden? Begründen sie die Antwort. <br>
Kopieren sie dafür den obenstehnden Code Block in den folgenden Code Block und ändern sie dann die Werte für die Startkonzentration im Abschnitt Startkonzentrationen, der Wert unter P0 (lassen sie alles andere gleich).

In [ ]:
//hier code einfügen

Bitte hier die Antwort eingeben

Die Beobachtung zeigt das die aktuelle Differnzialgleichung nicht das darstellt was wir eigentlich zeigen wollen. Daher nehmen wir eine kleine Änderung am aktuellen Modell vor um den erwünschten Effekt zu erzielen. Die neue Differnzialgleichung lautet
$$ inlac' = \frac {dinlac} {dt}= \alpha + \beta * \frac {inlac^{2}}{(K_{D})^{2} + inlac^{2}} - \gamma*inlac$$

Aufgabe 4.3: <br>
Setzen sie die Gleichung für intrazelluläre Lactosekonzentration, in der der Teil der postiven Autoregulation hoch 2 genommen wird, in die nachfolgende Simulation ein (Bitte beachten sie, dass sie die Lactosekonzentration als "inlac" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\alpha$, $\beta \textrm{ und } \gamma$ als "alpha", "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration

        //Differenzialgleichung die diese Änderung beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha + beta * (inlac**2.)/((inlac**2.) + (K_d**2.)) - (gamma*inlac)

        [| inlac' |]

//Startkonzentrationen
let P0 = [| 
    0.1
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = Ps |> SolPoints.toPoints 1 
    lactose |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe 4.4: <br>
Was beobachten sie?<br>




Hier die Antwort eingeben

Aufgabe 4.5: <br>
Erhöhen sie jetzt Schrittweise den Wert von der $Lactosekonzentration(0)$, nutzen sie dabei die folgenden Werte:

Notieren sie die Werte die sie für die Gleichgewichtspunkte finden, bei jeder Konzentration (der Wert bei dem sich die Konzentration nicht mehr merklich ändert)
$$0.5$$
$$1.0$$
$$1.5$$
$$2.0$$
$$2.5$$
$$3.0$$
$$3.5$$
$$4.0$$
$$5.0$$
$$6.0$$
$$7.0$$
$$8.0$$

Was können sie im Bezug auf das $Lactosekonzentration{GGW}$ sagen?
Kopieren sie dafür den obenstehnden Code Block in den folgenden Code Block und ändern sie dann die Werte für die Startkonzentration im Abschnitt Startkonzentrationen, der Wert unter P0 (lassen sie alles andere gleich).

Hier die Werte für die einzelnen $Lactosekonzentration(0)$ eintragen:




Anschliesend bitte beantworten was sie über das  $Lactosekonzentration{GGW}$ im allgemeinen sagen können.

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen.

Aufgabe 4.6: <br>
Bestimmen sie ob es sich um stabile oder instabile Punktattraktoren handelt. Nutzen sie dafür die Werte die sie oben eingetragen haben.
Stabil bedeutet das die $Lactosekonzentration{GGW}$ vom folgenden und vorherigen $Lactosekonzentration(0)$ identisch sind. Als Beispiel (diese Werte sind ausgedacht und sollen nur als Beispiel dienen):
$Lactosekonzentration(0)$ wird zu $Proteinkonzentration_{GGW}$
$$Lactosekonzentration(1.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(2.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(3.0) -> Lactosekonzentration{GGW} = 2$$

dann handelt es sich beim Punktattraktor 2 um einen stabilen Punktattraktor. 

Instabil bedeutet analog das die beiden flannkierenden Proteinkonzentration_{GGW} zu einem anderen Punktattraktor wandern. Als Beispiel:

$$Lactosekonzentration(3.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(4.0) -> Lactosekonzentration{GGW} = 4$$
$$Lactosekonzentration(5.0) -> Lactosekonzentration{GGW} = 8$$

Hier wäre der Punktattraktor bei 4 instabil. 

Gehen sie jezt bei denen von ihnen bestimmten Punkten anlog vor (es reicht wenn sie schreiben Punktattraktor X ist stabil/instabil).

Hier die Antwort eingeben.

Ein System das zwei stabile Punktattraktoren hat wird auch bistabiles System genannt. **Bistabilität** ist eine Schlüsseleigenschaften von biologischen Systeme, da das System auch dann an einem der beiden Punktattraktoren verweilt, wenn es leichte Störungen oder es leicht veränderte äußere Einflüße gibt. Das bedeutet bezogen auf das lac Operon, das dieses nur dann expremiert wird wenn es eine hohe Konzentration an extrazellulärere Lactose gibt. Bei nur sehr geringen Mengen kommt es nicht zur Ausbilding von Allolactose und das lac Operon wird weiterhin durch lacI inhibiert. Dies hat den Vorteil das die Energie die auf die Produtkion von lacY, laxZ, und lacA verwendet werden würde gespart wird, da die Menge an Lactose dies nicht rechtfertigt. Auf der anderen Seite kommt es bei einer hoch genugen Lactosekonzentration zur Ausbildung von genug Allolactose, so das das Lac Operon nicht mehr durch LacI inhibiert wird. Da diese Repression sehr stark ist, der Faktor ist ungefähr 1000, kommt es anschliesend zu einer sehr starken expression des Lac Operon. <br>
Die Fähigkeit nur dann Gene zu expremieren, wenn sie wirklich gebraucht werden, aber dann in sehr hohem Maße, ist ein grundlegender Vorteil gegenüber einem Organismus der diese Fähigkeit nicht hat und immer alles mit der selben Rate produziert. Daher hat sich **Bistabilität** als eines der Grundlegenden Bausteine in der Evolution durchgesetzt und kann heute in jedem Organismus bei fast allen relevanten Genzyklen beobachtet werden.
Bistabilität ist allerdings nur eine spezialform der **Multistabilität**. Multistabilität bedeutet das ein System mehere Fixpunkte hat zu denen es wandern kann, anhängig von den äußeren Einflüßen. Dies hat beim System von Lactose noch keinen Vorteil, aber bei komplexeren Problemen wo wie der Prolieferation ist es deutlich Vorteilhafter. 


Aufgabe 4.7: <br>
Teilen sie Gleichung welche die Änderung der Lactosekonzentration im Bezug auf die Zeit beschreibt in zwei Gleichungen. Die erste soll die Zunahme der Lactose beschreiben, die zweite die Abnahame, also
$$ \frac {dLactosekonzentration} {dt} = \textrm{Zunahme der Lactosekonzentration} $$
$$ \frac {dLactosekonzentration} {dt} = \textrm{Verringerung der Lactosekonzentration}$$

Fügen sie die beiden Gleichungen in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{d}$ als K_d eingesetzt wird). 

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

//Graphische Darstellung der Gleichungen
[
    // Fügen sie die Gleichung für den Proteinabbau ein
    Chart.Line [ for x in 0.0 .. 0.1 .. 15.0 -> (inlac, (gamma*inlac)) ]
    |> Chart.withTraceInfo("Proteinabbau");
    //Fügen sie die Gleichung für die Proteinproduktion ein
    Chart.Line [ for x in 0.0 .. 0.1 .. 15.0 -> (inlac, alpha + beta * (inlac**2.)/((inlac**2.) + (K_d**2.))) ]
    |> Chart.withTraceInfo("Proteinproduktion");
]

|> Chart.combine
|> Chart.withXAxisStyle("Proteinkonzentration")
|> Chart.withYAxisStyle("Veränderung der Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Der gezeigte Graph stellt auf der Y-Achse die Änderung der Lactosekonzentration dar, also Import' und Abbau'. Die X-Achse repäsentiert die Lactosekonzentration. Diese graphische Darstellung macht es sehr einfach die Punktattraktionen zu bestimmen. Diese sind die Schnittpunkte der beiden Kurven, da bei diesen der Import und der Verbrauch den gleichen Wert besitzen. In diesem Fall befinden sich die Schnittpunkte der beiden Kurven bei 1, 2.5 und 7.3 .<br>
Diese Art von Graph ermöglicht es auch zu bestimmen ob es sich um stabile oder instabile Punktattraktionen handelt. Dafür muss man sich nur überlegen, wohin die Lactosekonzentration wandert. <br>
Einige Beispiele folgen: <br>
$$Lactosekonzentration (0) = 1$$ 
Dieser Wert liegt bereits auf einer Punktattraktion und daher wird die Lactosekonzentration sich nicht ändern.
$$Lactosekonzentration (0) = 2$$ 
Bei dieser Lactosekonzentration ist der Wert von der Kurve für die Verringerung höher als der für den Import. Das heißt es wird mehr Lactose verbraucht als importiert, somit sinkt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 1.
$$Lactosekonzentration (0) = 2.5$$
Dieser Wert liegt bereits auf einer Punktattraktion und daher wird die Lactosekonzentration sich nicht ändern. 
$$Lactosekonzentration (0) = 3$$
Bei dieser Lactosekonzentration ist der Wert von der Kurve für den Import höher als der für den Verbrauch. Das heißt es wird mehr Lactose importiter als verbraucht, somit steigt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 7,3.
$$Lactosekonzentration (0) = 10$$
Bei dieser Lactosekonzentration ist der Wert von der Kurve für den Verbrauch höher als der für den Import. Das heißt es  wird mehr Lactose verbraucht als importiert, somit sinkt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 7.3.

Das heißt die Punktattraktion 1 ist stabil, da höhere Werte sich zu diesem Wert hinbewegen (bis die nächste Punktattraktion erreicht wird).
Die Punktattraktion 2.5 ist instabil, da höhere Werte zur nächsten Punktattraktion, 7.3, wandern und niedrigere zur vorherigen Punktattraktion, 0. Nur wenn die $Lactosekonzentration(0) = 2.5$ kommt es zu diesem Punkt.

Aufgabe 4.8: <br>
Kopieren sie den obenstehnden code block und fügen sie diesen in den folgenden Code Block ein. 

In [ ]:
//Code block einfügen

Varrieren sie die Werte für die Abbaukonsante $\gamma$. Bestimmen sie die Punktattraktionen und ob diese stabil oder instabil sind. Notieren sie die Werte im nachfolgennden Block. Varrieren sie $\gamma$ von 0.7 bis 1.5 in 0.1 schritten (also 0.7,0.8.0.9,...).

Notieren sie hier die Werte


Aufgabe 4.9: <br>
Setzen sie jetzt die Werte in den nachfolgenden Code Block ein und führen sie diesen anschliesend aus.
Beachten sie dabei das folgende:
x Werte sind die Werte die sie für die Abbaukonstante $\gamma$ haben, y Werte sind die von ihnen notierten Punktattraktionen. 

In [ ]:
let x = [0.7; 0.8; 0.9; 1.0; 1.1;0.9; 1.0; 1.1; 0.9; 1.0; 1.1; 1.2]
let y = [7. ; 7. ; 7. ;7. ; 7.  ; 2.5; 2.5; 2.5 ; 1. ; 1.; 1. ; 1.  ]

Chart.Spline (x, y)



<!-- Plotly chart will be drawn inside this DIV -->

Dieser Graph ist ein sogenanntes **Bifurkationsdiagramm**. **Bifurkation** beschreibt die qualtitative Änderung des Zustandes eines Systems, das durch nicht lineare Gleichungen beschrieben wird, in Abhängigkeit der Veränderung eines Parameters der Teil der beschreibenden Gleichung ist. In diesem Fall haben sie den Parameter $\gamma$ verändert und es kam zu einer sogenannten **Saddle-Node Bifurkation**. Bei dieser Bifurkation verliert ein System zwei Punktattraktionen wenn der Bifurkationparameter einen bestimmten Wert erreicht. Dabei wandern die beiden Punktattraktionen aufeinander zu, wenn sich der Bifurkationparameter diesen Wert nähert und voneinander weg wenn er sich von diesem Wert entfernt. Das heißt das System wechselt zwischen den beiden Konformationen **bistabil** und **monostabil**, weshalb von einer qualtiativen Änderung gesprochen wird.

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen.

### Hill Funktion und Ultrasensitivität

Die Umgebung von einer Zelle ändert sich dauerhaft, da sich die Konzentrationen von allen Molekülen um die Zelle dauerhaft ändern. Durch aktiven oder passiven Import der Zelle, oder benachbarter Zellen, durch Export von bestimmten Molekülen, oder durch Diffusion und einige Möglichkeiten zu nennen. Die Zelle muss konstant die Expression von Genen und damit auch die Produktion von Proteinen an diese Veränderungen anpassen. Wie beim lac Operon gezeigt ist dies in vielen Fällen nicht möglich mit den am Vortrag besprochenen Differenzialgleichungen. Dies liegt daran das diesen Prozessen kein linearer Zusammenhang zu Grund liegt, sondern ein **ultrasensitives** Verhalten.
Das heißt das ein Prozess nur dann stattfindet wenn ein bestimmter Schwellenwert überschritten wird, während es davor keinen Effekt gibt. Oft ist dieses ultrasensitive Verhalten bedingt durch **Kooperativität**. Beim lac Operon führte dieses ultrasensitve Verhalten dazu das es nur bei hoch genugen Anfangskonzentration zu einem Verstärkten Import von Lactose kam. Allerdings musste dies bewekstelligt werden durch das Einführen eines Faktors hoch 2. Dieser Faktor beschreibt allerdings nicht alle Möglichen Prozesse und ist auch nicht sehr allgemein. Daher wird für die mathematische Beschreibung von solchen Ultrasensitiven Prozessenin der Biologie zumeist die Hillfunktion verwendet. Das anwenden von dieser auf die bisher besprochenen negative Autoregulation führt zu der folgenen Differenzialgleichung 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ (\frac {Proteinkonzentration}{K_{D}})^{n}}  - \gamma*Proteinkonzentration$$

die Gleichung für eine ultrasentives Verhalten von einer Proteinproduktion die durch einen Repressor beeinflusst wird ist identisch, nur müssen sie die Proteinkonzentration im Term, der $\beta$ beeienflusst, durch die Repressorkonezntration ersetzten. Die Differenzialgleichung für postive Autoregulation ist 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {Proteinkonzentration^{n}}{{(K_{D})^{n}} + Proteinkonzentration^{n}} - \gamma*Proteinkonzentration$$
auch hier führt das ersetzten der Protienkonzentration, im $\beta$ beeinflussenden Term, mit der Aktivatorkonezentration zu der Gleichung für eine Proteinproduktion die von diesem beieinflusst wird. 
Der Paramerter $n$ ist der Hillkoeffizent und beschreibt wie stark das ultrasensitve Verhalten ist. Ein Wert von 1 beschreibt die bisher besprochenen Verhaltensweisen, während ein Wert von 2 oder höher das ulatrsensitive Verhalten beschreibt. 
Im nachfolgenden soll überprüft werden, ob die Hill Funktion wirlich so wirkt wie besprochen.

Aufgabe 4.10:

Setzen sie die Gleichung für postive Autoregulation in die nachfolgende Simulation ein, dabei sollte die postive Autoregulation durch eine Hill Funktion beschrieben werden (Bitte beachten sie, dass sie die Proteinkonznetration als "Proteinkonznetration2" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      2.0  // Hill Koeffizent 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonznetration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonznetration2 = P[1] // Änderung in der Proteinmenge
        //Änderung der Proteinkonzentration ohne positive Autoregulation
        let Proteinkonznetration1' = beta - (gamma*Proteinkonznetration1)
        // Änderung des Proteinkonzentration bei positiver Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonznetration2' = beta * (Proteinkonznetration2**n)/((Proteinkonznetration2**n) + (K_d**n)) - (gamma*Proteinkonznetration2)

        [| Proteinkonznetration1'; Proteinkonznetration2' |]

//Startkonzentrationen
let P0 = [| 
    //Startkonzentration für das unregulierte Protein (muss nicht verändert werden)
    0.0 
    //Startkonzentration für das durch postive Autoreguliation regulierte Protein (hier bitte die Startwerte ändern)
    4.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("negative AR");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

Die Konstanten für den gezeigten Graphen sind:
$$\beta = 10$$
$$\gamma= 1$$
$$n = 4$$
$$K_{D} = 1$$
$$Proteinkonzentration (0) = 0.1$$
Aufgabe 4.11: <br>
Welchen Effekt hat postive Autoregulation bei einem Wert von $n=1$ auf die Antwortszeit?


Hier die Antwort eintippen

Aufgabe 4.12 <br>
Was passiert bei $n=2$?
Kopieren sie die obenstehnden code block für die postive Autoregulation und fügen sie diesen in den folgenden Code Block ein. Verändern sie dann die Hill Konstante. 

Was beobachten sie?

Hier die Antwort eintippen

Es folgt eine kurze Besprechung.<br>


------------------

### Osszillation/Hopf Bifukration folgt